Special composition objects

In [27]:
try:
    from .composites import composite, a
except:
    from composites import composite, a
from functools import partialmethod
__all__ = tuple()

from collections import UserDict, UserList
from toolz import isiterable
dunder = '__{}__'.format
__all__ = 'cache', 'persist'

In [28]:
class parallel(composite):
    """An embarassingly parallel composite
    
    All map functions are delayed
    >>> parallel(jobs=4)[range].map(print) # doctest: +SKIP
    """
    def __init__(self, jobs=4, data=None):
        setattr(self, 'jobs', jobs) or super().__init__(data)
        
    def map(self, function):
        """A delay each function."""
        return super().__getattr__('map')(__import__('joblib').delayed(function))
    
    def __call__(self, *args, **kwargs):
        return __import__('joblib').Parallel(self.jobs)(super().__call__(*args, **kwargs))
        
    __truediv__ = map

In [29]:
from collections import OrderedDict

In [30]:
class dispatch(OrderedDict):
    """a singledispatching composite

    >>> f = dispatch((str, str.upper), (int, range), (object, type))
    >>> f('text'), f(42), f({10})
    ('TEXT', range(0, 42), <class 'set'>)
    """
    def __init__(self, *data):
        super().__init__(
            data[0] if len(data) is 1 and isinstance(data[0], (list, dict)) else data)
    def __call__(self, *args, **kwargs):
        for condition, object in self.items():
            if callable(condition) and not isinstance(condition, type):
                if condition(*args, **kwargs): break
            else:
                if all(isinstance(arg, type) for arg, type in zip(
                    args, condition if isiterable(condition) else (condition,)
                )): break
        else:
            raise TypeError("No conditions matched for {}.".format(args))
            
        return object(*args, **kwargs)

In [10]:
class cache(UserDict):
    """cache on the first argument, use star to store tuples.
    
    >>> c = cache(range)
    >>> assert c(10)(20)(30) and 10 in c and 20 in c and 30 in c
    """
    def __init__(self, callable, data=None):
        self.callable = callable or composition()
        super().__init__(data)
        
    def __missing__(self, item):
        self(item) if not isinstance(item, tuple) else self(*item)
        return self[item]
        
    def __call__(self, *args, **kwargs): 
        if args[0] not in self:  self[args[0]] = self.callable(*args, **kwargs)
        return self

In [11]:
class persist(__import__('shelve').DbfilenameShelf):
    """
    >>> c = cache(range)
    >>> assert c(10)(20)(30) and 10 in c and 20 in c and 30 in c
    >>> p = persist('test', 'n')
    >>> assert not list(p.keys()) and p(40) and 40 in p.keys() and 30 not in p.keys()
    >>> p.update(c)
    >>> assert 30 in p
    >>> p.close()
    >>> p2 = persist('test', 'r')
    >>> assert 10 in p2 and 20 in p2 and 30 in p2 and 40 in p2
    """

    def __init__(self, callable, *args, **kwargs):
        if isinstance(callable, str): args = callable, *args
        super(persist, self).__init__(*args, **kwargs)
        if isinstance(callable, str): callable = self.get(callable, composition())
        self.callable = callable
        
    def close(self):
        try:
            self[callable] = self.callable
            while hasattr(self[callable], 'callable'):  self[callable] = self[callable].callable
        except:
            pass
        return super().close()
    
    def __method__(self, method, item, *args):
        return getattr(super(), dunder(method))(str(item), *args)
    
    __getitem__ = partialmethod(__method__, 'getitem')
    __setitem__ = partialmethod(__method__, 'setitem')
    __contains__ = partialmethod(__method__, 'contains')
    
    def __call__(self, arg, **kwargs): 
        if arg not in self:
            self[arg] = self.callable(arg, **kwargs)
        return self

In [12]:
if __name__ == '__main__':
    print(__import__('doctest').testmod(verbose=False))
    !jupyter nbconvert --to python --TemplateExporter.exclude_input_prompt=True objects.ipynb

**********************************************************************
File "__main__", line ?, in __main__.dispatch
Failed example:
    f('text'), f(42), f({10})
Exception raised:
    Traceback (most recent call last):
      File "/Users/tonyfast/anaconda/lib/python3.5/doctest.py", line 1321, in __run
        compileflags, 1), test.globs)
      File "<doctest __main__.dispatch[1]>", line 1, in <module>
        f('text'), f(42), f({10})
      File "<ipython-input-9-3244740b67b8>", line 14, in __call__
        self.dispatch = __import__('functools').singledispatch(λ[:])
    NameError: name 'λ' is not defined
**********************************************************************
File "__main__", line ?, in __main__.persist
Failed example:
    p = persist('test', 'n')
Exception raised:
    Traceback (most recent call last):
      File "/Users/tonyfast/anaconda/lib/python3.5/doctest.py", line 1321, in __run
        compileflags, 1), test.globs)
      File "<doctest __main__.persist[2]>", l